In [1]:
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
import json
%matplotlib widget

In [2]:
## Plot spectra for both
 ## WORK IN PROGRESS
sys.path.append('../../data/')
from Ba133_data_AV_analysis import * 

#load data, cuts and calibration
data_path = "/lfs/l1/legend/detector_char/enr/hades/char_data/I02160A/tier2/ba_HS4_top_dlt/pygama/v00.00/"
with open("../../data/detectors/I02160A/calibration_coef.json") as json_file:
    calibration_coefs = json.load(json_file)
    m = calibration_coefs['m']
    m_err = calibration_coefs['m_err']
    c = calibration_coefs['c']
    c_err = calibration_coefs['c_err']

df_data = read_all_dsp_h5(data_path, cuts=False)
e_ftp_data = df_data['e_ftp']
energy_data = (e_ftp_data-c)/m

passed_cuts = json.load(open('/lfs/l1/legend/users/aalexander/large_files/cuts/I02160A_ba_top_passed_cuts.json','r')) #passed cuts
df_data_cuts = read_all_dsp_h5(data_path, cuts=True, passed_cuts=passed_cuts)
e_ftp_data_cuts = df_data_cuts['e_ftp']
energy_data_cuts = (e_ftp_data_cuts-c)/m

#load MC
MC_path = "/lfs/l1/legend/users/aalexander/hdf5_output/processed/"
MC_file_ID = "IC160A_ba_top_81mmNEW8_01_newresolution"
df_MC_FCCD0744 =  pd.read_hdf(MC_path+"processed_detector_"+MC_file_ID+"_FCCD0.744mm.hdf5", key="procdf")
energy_MC_FCCD0744 = df_MC_FCCD0744['energy']*1000
df_MC_FCCD0698 =  pd.read_hdf(MC_path+"processed_detector_"+MC_file_ID+"_FCCD0.698mm.hdf5", key="procdf")
energy_MC_FCCD0698 = df_MC_FCCD0698['energy']*1000
print(energy_MC_FCCD0698)
print(energy_MC_FCCD0744)

#scale MC
with open('../detectors/I02160A/IC160A_ba_top_81mmNEW8_01_newresolution_FCCD0.744mm_dlt_observables.json') as json_file:
    dlt_observables = json.load(json_file)
    R_simdata_356_FCCD0744 = dlt_observables['R_simdata_356_counts']
    print(R_simdata_356_FCCD0744)
        
with open('../detectors/I02160A/IC160A_ba_top_81mmNEW8_01_newresolution_FCCD0.698mm_dlt_observables_cuts.json') as json_file:
    dlt_observables = json.load(json_file)
    R_simdata_356_FCCD0698 = dlt_observables['R_simdata_356_counts']
    print(R_simdata_356_FCCD0698)


bins = np.arange(0,450,0.15)
fig, ax = plt.subplots()
counts_data, bins, bars = plt.hist(energy_data, bins=bins,  label = "Data", histtype = 'step', linewidth = '0.35')
counts_data_cuts, bins, bars = plt.hist(energy_data_cuts, bins=bins,  label = "Data (cuts)", histtype = 'step', linewidth = '0.35')
counts_MC_FCCD0744, bins, bars = plt.hist(energy_MC_FCCD0744, bins = bins, weights=(1/R_simdata_356_FCCD0744)*np.ones_like(energy_MC_FCCD0744), label = "MC: FCCD 0.744mm (scaled)", histtype = 'step', linewidth = '0.35')
counts_MC_FCCD0698, bins, bars = plt.hist(energy_MC_FCCD0698, bins = bins, weights=(1/R_simdata_356_FCCD0698)*np.ones_like(energy_MC_FCCD0698), label = "MC: FCCD 0.698mm (scaled)", histtype = 'step', linewidth = '0.35')
plt.xlabel("Energy [keV]")
plt.ylabel("Counts")
plt.xlim(0, 450)
plt.yscale("log")
plt.legend(loc = "lower left")
plt.show()


0          302.944951
1          312.467729
2          248.154367
3          356.269414
4          355.416232
              ...    
4063842    355.814522
4063843    356.316063
4063844    249.273861
4063845    355.728685
4063846    356.293823
Name: energy, Length: 4063847, dtype: float64
0          302.426463
1          312.361570
2          247.541278
3          356.910759
4          355.760823
              ...    
4023430    356.735169
4023431    356.045003
4023432    249.073528
4023433    356.295695
4023434     89.248756
Name: energy, Length: 4023435, dtype: float64
3.1255920187062203
4.326534799842924


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [3]:
## Calculate absolute data/MC

#read in csv files
data_MC_ratios = "../detectors/I02160A/IC160A_ba_top_81mmNEW8_01_newresolution_FCCD0.744mm_DataMCRatios.csv"
data_MC_ratios_cuts = "../detectors/I02160A/IC160A_ba_top_81mmNEW8_01_newresolution_FCCD0.698mm_DataMCRatios_cuts.csv"


with open('../detectors/I02160A/IC160A_ba_top_81mmNEW8_01_newresolution_FCCD0.744mm_dlt_observables.json') as json_file:
    dlt_observables = json.load(json_file)
    R_simdata_356_FCCD0744 = dlt_observables['R_simdata_356_counts']
    print(R_simdata_356_FCCD0744)
        
with open('../detectors/I02160A/IC160A_ba_top_81mmNEW8_01_newresolution_FCCD0.698mm_dlt_observables_cuts.json') as json_file:
    dlt_observables = json.load(json_file)
    R_simdata_356_FCCD0698 = dlt_observables['R_simdata_356_counts']
    print(R_simdata_356_FCCD0698)


ratios_df = pd.read_csv(data_MC_ratios)
ratios = ratios_df['ratio']
ratios_scaled = ratios/R_simdata_356_FCCD0744
ratios_err = ratios_df['ratio_err']
ratios_err_scaled = ratios_err/R_simdata_356_FCCD0744
energies = ratios_df['bin']


ratios_cuts_df = pd.read_csv(data_MC_ratios_cuts)
ratios_cuts = ratios_cuts_df['ratio']
ratios_cuts_scaled = ratios_cuts/R_simdata_356_FCCD0698
ratios_err_cuts = ratios_cuts_df['ratio_err']
ratios_err_cuts_scaled = ratios_err_cuts/R_simdata_356_FCCD0698
energies_cuts = ratios_cuts_df['bin']

#plot
plt.figure()
plt.plot(energies, ratios, 'o', ms=1.25,label = "FCCD: 0.744mm, no cuts on data", color = 'orange')
plt.plot(energies_cuts, ratios_cuts, 'o', ms =1.25, label = "FCCD: 0.698mm, cuts on data", color = 'green')
ones = [1]*len(energies)
plt.plot(energies, ones, "k-.")
plt.xlabel('Energy (keV)')
plt.ylabel('Data/MC')
plt.yscale("log")
plt.ylim(0.01,500)
plt.xlim(0,450)
plt.legend()
plt.show()






#this is incorrect - will have been scaled twice
# plt.figure()
# plt.plot(energies, ratios_scaled, 'o', ms=1.25,label = "FCCD: 0.744mm, no cuts on data", color = 'orange')
# plt.plot(energies_cuts, ratios_cuts_scaled, 'o', ms =1.25, label = "FCCD: 0.698mm, cuts on data", color = 'green')
# ones = [1]*len(energies)
# plt.plot(energies, ones, "k-.")
# plt.xlabel('Energy (keV)')
# plt.ylabel('Data/MC_{scaled}')
# plt.yscale("log")
# plt.ylim(0.01,500)
# plt.xlim(0,450)
# plt.legend()
# plt.show()




3.1255920187062203
4.326534799842924


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [4]:
## Calculate data/MC for peak counts only

peaks = [79,81,161, 223, 276, 303, 356, 383]
energies_peaks = [79.6, 81, 161, 223, 276, 303, 356, 383 ] #truth values keV

peak_counts_data = []
peak_counts_data_err = []
with open('../../data/detectors/I02160A/dlt_observables.json') as json_file:
    dlt_observables = json.load(json_file)
    for index, peak in enumerate(peaks):
        C = dlt_observables['C_'+str(peak)]
        peak_counts_data.append(C)
        C_err = dlt_observables['C_'+str(peak)+'_err']
        peak_counts_data_err.append(C_err)
        
peak_counts_data_cuts = []
peak_counts_data_cuts_err = []
with open('../../data/detectors/I02160A/dlt_observables_cuts.json') as json_file:
    dlt_observables = json.load(json_file)
    for index, peak in enumerate(peaks):
        C = dlt_observables['C_'+str(peak)]
        peak_counts_data_cuts.append(C)
        C_err = dlt_observables['C_'+str(peak)+'_err']
        peak_counts_data_cuts_err.append(C_err)

peak_counts_MC_FCCD0744 = []
peak_counts_MC_FCCD0744_err = []
with open('../detectors/I02160A/IC160A_ba_top_81mmNEW8_01_newresolution_FCCD0.744mm_dlt_observables.json') as json_file:
    dlt_observables = json.load(json_file)
    R_simdata_356_FCCD0744 = dlt_observables['R_simdata_356_counts']
    print(R_simdata_356_FCCD0744)
    for index, peak in enumerate(peaks):
        C = dlt_observables['C_'+str(peak)]
        C = C/R_simdata_356_FCCD0744
        peak_counts_MC_FCCD0744.append(C)
        C_err = dlt_observables['C_'+str(peak)+'_err']
        C_err = C_err/R_simdata_356_FCCD0744
        peak_counts_MC_FCCD0744_err.append(C_err)
        
peak_counts_MC_FCCD0698 = []
peak_counts_MC_FCCD0698_err = []
with open('../detectors/I02160A/IC160A_ba_top_81mmNEW8_01_newresolution_FCCD0.698mm_dlt_observables_cuts.json') as json_file:
    dlt_observables = json.load(json_file)
    R_simdata_356_FCCD0698 = dlt_observables['R_simdata_356_counts']
    print(R_simdata_356_FCCD0698)
    for index, peak in enumerate(peaks):
        C = dlt_observables['C_'+str(peak)]
        C = C/R_simdata_356_FCCD0698
        peak_counts_MC_FCCD0698.append(C)
        C_err = dlt_observables['C_'+str(peak)+'_err']
        C_err = C_err/R_simdata_356_FCCD0698
        peak_counts_MC_FCCD0698_err.append(C_err)



ratios_counts = np.array(peak_counts_data)/np.array(peak_counts_MC_FCCD0744)
ratios_counts_err = ratios_counts*(np.sqrt((np.array(peak_counts_data_err)/np.array(peak_counts_data))**2 + (np.array(peak_counts_MC_FCCD0744_err)/np.array(peak_counts_MC_FCCD0744))**2))

ratios_counts_cuts = np.array(peak_counts_data_cuts)/np.array(peak_counts_MC_FCCD0698)
ratios_counts_cuts_err = ratios_counts_cuts*(np.sqrt((np.array(peak_counts_data_cuts_err)/np.array(peak_counts_data_cuts))**2 + (np.array(peak_counts_MC_FCCD0698_err)/np.array(peak_counts_MC_FCCD0698))**2))


fig, ax = plt.subplots()
plt.errorbar(energies_peaks, ratios_counts, xerr=0, yerr =ratios_counts_err, label = "MC: FCCD 0.744mm", elinewidth = 1, fmt='o', ms = 3, mew = 2.5 ,color = 'orange')
plt.errorbar(energies_peaks, ratios_counts_cuts, xerr=0, yerr =ratios_counts_cuts_err, label = "Data with cuts, MC:FCCD: 0.698mm", elinewidth = 1, fmt='o', ms = 3, mew = 2.5, color = 'green')
x_ones = np.linspace(0,450,1000)
ones = [1]*len(x_ones)
plt.plot(x_ones, ones, "k-.")
plt.xlabel('Energy (keV)')
plt.ylabel('Data/MC_{peak counts}')
#plt.yscale("log")
#plt.ylim(0.1,10)
#plt.ylim(0,2)
plt.xlim(0,450)
plt.legend()
plt.show()

3.1255920187062203
4.326534799842924


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …